## Тестовое задание 2 от НРА
### Копчев Владислав

### Введение

В данной работе я решу задачу автоматического разбиения текста на предложения. Среди задач информационного поиска хорошо изучена задача исправления ошибок в словах, что полезно при решении проблем, появляющихся при запросах из не более чем одного предложения. Однако при работе с базами документов, книг, статей очень часто возникает необходимость делать поисковые запросы, содержащие несколько последовательных предложений — и искать не все документы, которые содержат слова из этих предложений, а те документы, которые содержат именно эти предложения целиком (в Яндексе, например, для такого запроса необходимо заключить последовательность предложений в кавычки). Именно здесь возникает данная задача, применение алгоритмов машинного обучения к которой еще не настолько хорошо изучена, как в задачи исправления ошибок в словах. При выполнении задачи мною был разработан алгоритм, который на вход принимает путь до текстового файла с документом, а на выходе возвращает этот текст с корректным разбиением на предложения. 

У данного алгоритма есть несколько основных приложений. Во-первых, это облегчит работу редакторов — ведь редакторы тоже ошибаются, и $F$-мера вектора их разбиения текста на предложения при сопоставлении с вектором правильного разбиения может оказаться даже ниже, чем у алгоритма машинного обучения. Таким образом, алгоритм может помочь и менеджерам — ведь даст им обоснования для решения о сокращении штата редакторов, что позволит уменьшить расходы компании. Этим же алгоритм может помочь и экспертам-юристам — при чтении неграмотно набранного текста они могут запустить этот алгоритм и уже через минуту получить более пригодную для чтения версию этого текста. Также алгоритм может помочь пользователям информационного поиска — он поможет сделать жизнь пользователей, которым необходимо писать запросы из нескольких предложений, лучше, о чем говорилось в самом начале статьи. Соответственно, и разработчики поиска смогут понять, какой алгоритм лучше использовать для решения данной проблемы. Помочь алгоритм может и разработчикам нейросетей: проверить корректность табличных данных с числовыми признаками гораздо легче, чем корректность неструктурированного текста. Соответственно, этот алгоритм позволит улучшить данные, доступные NLP-специалистом, что позволит их алгоритмам работать лучше. И, наконец, алгоритм позволит решить ряд аналитических задач, которые невозможно решить без него: можно взять набор запросов, состоящих из нескольких предложений, пользователей системы КонсультантПлюс и понять, как выглядят статистические свойства ошибок польщователей, что позволит получить большое количество полезной информации о пользователях. Также можно посмотреть, в каком отделе редакторы плохо исправляют ошибки или из каких внешних источников в систему КонсультантПлюс поступают неграмотно набранные тексты, что позволит лучше оценивать эффективность сотрудников. Наконец, алгоритм имеет научную значимость, ведь показывают, что пунктуация текстов может иметь большое число интересных статистических закономерностей. 

Как следует из [статьи](https://www.dialog-21.ru/digests/dialog2008/materials/html/83.htm), на которую я буду опираться в ходе решения, при использовании простых эвристик точность доходит до 96%, а полнота — до 90%. Соответственно, алгоритм машинного обучения должен превзойти эти цифры,  а его $F$-мера должна быть не ниже $\frac{2 \cdot 0.96 \cdot 0.9}{0.96+0.9}$ $\approx$ 0.93. Если говорить про скорость работы, то при анализе документов ФАС было выяснено, что в них содержится не более 300 терминальных знаков препинания, а сами тексты не содержат очень большого числа символов. Соответственно, нет особого смысла при таких значениях длины входных данных описывать требования к асимптотическим оценкам работы алгоритма по времени в зависимости от размера входных данных. При таких входных данных гораздо большее значение будет иметь не асимптотическая сложность алгоритмов, которые использованы в решении, а то, насколько они хорошо оптимизированы их разработчиками. Соответственно, лучше всего поставить ко времени работы алгоритма следующее требование: алгоритм должен опираться на наиболее эффективно реализованные библиотеки для работы с алгоритмами машинного обучения, регулярными выражениями, неструктурированным текстом и т. д. Как будет видно в дальнейшем, всем этим требованиям разработанный агоритм соответствует. 

Алгоритм можно улучшать большим количеством способов. На данный момень алгоритм имеет важное ограничение: модель машинного обучения была обучена на текстах постановлений ФАС и не будет давать хороший результат, например, на художественных текстах. Чтобы исправить этот недостаток, можно добавить в обучающую выборку другие тексты. Также можно разработать новые признаки, которые повысят эффективность алгоритма. Можно добавить другие знаки пунктуации в качестве претендентов на конец предложения, что позволит исправлять больше ошибок в текстах. Можно использовать различные более совершенные модели машинного обучения.

Ниже будет представлен описанный алгоритм и проведенные с ним эксперименты.

### Программа

Вкратце опишу принцип работы программы.

Для начала вводится название текстового файла с кодировкой `cp1251` (поскольку именно такую кодировку имели файлы постановлений ФАС). После этого текст размечается. Для выполнения этой задачи была написана функция приблизительно как в эксперименте 1 из [статьи](https://www.dialog-21.ru/digests/dialog2008/materials/html/83.htm). Сначала формируем множества претендентов-предложений `possible_sentences` и претендентов-знаков `possible_ends_list`, записываем псевдослова в список `left_pseudowords_list`, `left_pseudowords_list`, после чего формируем таблицу признаков `possible_ends_df`. Далее идет загрузка предобученной модели (градиентный бустинг для классификации с подобранными с помощью `GridSearchCV` гиперпараметрами) и по предсказанным ей значениям восстановливается текст, в котором после каждого претендента-знака, который является концом предложения, вставляется символ перевода строки, а у следующего предложения убираются пробелы в начале — то есть, каждое предложение расположено на новой строке. Для работы программы необходим доступ к 3 файлам, содержащим заранее сформированные список сокращений, предобученную модель и предобученный OneHotEncoder.

О терминологии:
- Псевдослова — любая последовательность букв, цифр или символов (за исключением `\n`, `\t`, букв не из английского или русского алфавита и некоторых других символов) до пробела или символа перевода строки
- Претенденты-предложения — любая последовательность псевдослов до терминального знака (`!`, `.`, `?`).
- Претенденты-знаки — терминальные знаки

О признаках:
- `sign`: терминальный знак
- `sign index`: его индекс в тексте
- `right`: ближайшее псевдослово справа (в нормальной форме)
- `left`: ближайшее псевдослово слева
- `dleft`: количество псевдослов слева
- `dright`: количество пседовлов справа
- `abbleft`: наличие ближайшего псевдослова слева в списке сокращений
- `abbright`: наличие ближайшего псевдослова справа в списке сокращений
- `cright`: список характеристик правого ближайшего псевдослова (Кириллица, Латиница, Числа, Большая/маленькая заглавная буква)
- `cleft`: список характеристик левого ближайшего псевдослова (Кириллица, Латиница, Числа, Большая/маленькая заглавная буква)
- `wright`: ближайшее псевдослово справа, содержащее не меньше 1 буквы или цифры
- `cwright` количество таких псевдослов справа
- `end?`: целевая переменная, 1 — конец предложения, 0 — нет

Для составления списка сокращений воспользуемся корпусом OpenCorpora.

Также я добавил собственные признаки:

- `left_caps`: набрано ли левое псевдослово только заглавными буквами?
- `right_caps`: аналогично про правое

Итак, чтобы запустить программу, необходимо запустить следующую функцию:

In [18]:
start()

Введите название текстового файла с постановлением ФАС Уральского округа


 raw_text/Постановление ФАС Уральского округа от 05_06_2006 N Ф09-4478.txt


г.Ф09-4478/06-С3

Федеральный арбитражный суд Уральского округа в составе председательствующего Гайдука А.А., судей Черкасской Г.Н., Купреенкова В.А.рассмотрел в судебном заседании кассационную жалобу открытого акционерного общества "Уралсвязьинформ" (далее - общество) на решение суда первой инстанции от 27.01.2006 (резолютивная часть от 24.01.2006) и постановление суда апелляционной инстанции от 30.03.2006 Арбитражного суда Свердловской области по делу N А60-34123/2005-С3.

В судебном заседании принял участие представитель общества - Соломеин Е.А.доверенность от 22.12.2005).

Представитель государственного унитарного предприятия "Уралэнергочермет-ДС" (далее - предприятие), надлежащим образом извещенного о времени и месте судебного разбирательства, в судебное заседание не явился.


Предприятие обратилось в Арбитражный суд Свердловской области с иском к обществу об обязании выплатить компенсацию за долю в общей долевой собственности на не завершенный строительством II производственный к

### Эксперименты

Для тестирования работы алгоритма я разметил 20 текстов постановлений ФАС (для удобства разметки была написана отдельная функция) и для каждого размеченного документа сформировал тестовую выборку, состояющую из него самого, и обучающую из остальных 19 — ведь именно так алгоритм и будет использоваться, он будет обучен на всех размеченных текстах постановлений ФАС и будет принимать какой-то один другой текст на вход. Посмотрим на $F$-меру при использовании бустинга:

In [41]:
test()

1 / 20 0.961038961038961
2 / 20 1.0
3 / 20 0.9824561403508771
4 / 20 0.9824561403508771
5 / 20 1.0
6 / 20 0.9655172413793104
7 / 20 1.0
8 / 20 0.975609756097561
9 / 20 0.9866666666666666
10 / 20 1.0
11 / 20 0.9841269841269841
12 / 20 1.0
13 / 20 1.0
14 / 20 0.988235294117647
15 / 20 0.9876543209876543
16 / 20 1.0
17 / 20 0.9599999999999999
18 / 20 0.9555555555555556
19 / 20 0.9803921568627451
20 / 20 0.9859154929577464


Мы видим, что $F$-мера $\geq 0.95$.

Протестируем таким же способом другие алгоритмы:

In [48]:
test_other()

DecisionTreeClassifier
1 / 20 0.961038961038961
2 / 20 1.0
3 / 20 0.9824561403508771
4 / 20 0.9824561403508771
5 / 20 1.0
6 / 20 0.983050847457627
7 / 20 1.0
8 / 20 0.975609756097561
9 / 20 0.9866666666666666
10 / 20 1.0
11 / 20 0.9841269841269841
12 / 20 1.0
13 / 20 1.0
14 / 20 0.988235294117647
15 / 20 0.9876543209876543
16 / 20 1.0
17 / 20 0.9599999999999999
18 / 20 0.9555555555555556
19 / 20 0.9803921568627451
20 / 20 0.9859154929577464
LogisticRegression
1 / 20 0.961038961038961
2 / 20 1.0
3 / 20 0.9824561403508771
4 / 20 0.9824561403508771
5 / 20 1.0
6 / 20 0.9655172413793104
7 / 20 1.0
8 / 20 0.975609756097561
9 / 20 0.9866666666666666
10 / 20 1.0
11 / 20 0.9841269841269841
12 / 20 1.0
13 / 20 1.0
14 / 20 0.988235294117647
15 / 20 0.9876543209876543
16 / 20 1.0
17 / 20 0.9736842105263158
18 / 20 0.9555555555555556
19 / 20 0.9803921568627451
20 / 20 0.9714285714285714
RandomForestClassifier
1 / 20 0.961038961038961
2 / 20 1.0
3 / 20 0.9824561403508771
4 / 20 0.9824561403508771
5 

Итак, мы видим, что все алгоритмы со стандартными значениями гиперпараметров дают примерно одинаковое качество работы. Также в результате тюнинга гиперпараметров для SVC был сделан вывод, что SVC дает наилучший результат при `clf = svm.SVC(C=0.24, kernel='linear')`.

Помимо этого, был проведен еще один эксперимент, в ходе которого протестирован признак `left_length` — длина левого псевдослова. Он показал себя плохо, ухудшив $F$-меру для каждого алгоритма, что ожидаемо — вряд ли тут есть какая-то значимая корреляция с таргетом.

И, наконец, был проведен эксперимент, в ходе которого были размечены 1–12 примеры из статьи, чтобы проверить работу алгоритма вне документов. Результаты были удовлетворительные, однако при обучении бустинга было решено не включать эти документы в обучающую выборку.

### Выводы

Итак, был разработан алгоритм автоматического разбиения текста на предложения, удовлетворяющий поставленным требованиям по скорости и эффективности работы. Были проведены несколько экспериментов, описаны приложения алгоритма. 

### Приложение

1. Функция для составления списка сокращений:

In [ ]:
def save_abrs():
    # !wget http://opencorpora.org/files/export/annot/annot.opcorpora.xml.zip; open annot.opcorpora.xml.zip
    corpus = opencorpora.load('annot.opcorpora.xml')
    abr = corpus.sentences 
    abbreviatures = []

    for sent in abr:
        srch = re.findall('(^|\s)' + '[А-я]*' + '[.]+[ ]+[А-Я]+[А-я]*', sent.source)
        if srch:
            abbreviatures.append(sent.source)

    with open("abbrevs", "wb") as fp:
        pickle.dump(abbreviatures, fp)
    fp.close()
        
# save_abrs()

2. Функция для разметки текстов ФАС:

In [ ]:
def markup():
    import re
    import pickle

    filename = input()

    with open('raw_text/' + filename + '.txt',
             encoding='cp1251', errors='ignore') as fp:
        # считываем сразу весь файл; урааааа
        text = fp.read()
        fp.close()

    # text = 'Но ведь Маша знает А. Б. Иванова много лет и никогда про него ничего плохого  не слышала!!'

    possible_sentences = re.findall('[ЁёА-я0-9 ,()+\-/"@#№$%_=*;&^<>—:A-Za-z]*[.!?]', text)

    pattern = '[.!?]'
    possible_ends = []

    p = re.compile(pattern)

    ans = []

    for m in p.finditer(text):
        index = m.start()
        character = m.group()
        possible_ends.append(character)

        # print(possible_sentences)

    for i in range(len(possible_sentences)):
        cnt = 1
        if i > 0:
            print(possible_sentences[i-1], end='')
            cnt += 1
        print(possible_sentences[i], end='')
        if i < len(possible_sentences) - 1:
            print(possible_sentences[i+1])
        else:
            print()
        print(str(i+1) + '/' + str(len(possible_sentences)), sep='')
        print(str(cnt) + '-й по счету в данной строке ' + 'знак ' + possible_ends[i] + ' является концом или нет? (0/1)')
        cur_ans = int(input())
        ans.append(cur_ans)
        print()

    print(ans)

    with open('raw_text/' + filename + '_rzm', "wb") as fp:
        pickle.dump(ans, fp)
        fp.close()

    # save_abrs()

3. Функция для тестирования модели с бустингом:

In [39]:
from sklearn.ensemble import GradientBoostingClassifier

def find_abb(word, abbreviatures_list):
    abbreviatures = dict()
    for sent in abbreviatures_list:
        srch = re.findall('(^|\s)' + word + '[.]+[ ]+[А-Я]+[А-я]*', sent)
        if srch:
            return 1
    return 0

def make_table(text, answers):
    possible_sentences = re.findall('[ЁёА-я0-9 ,()+\-/"@#№$%_=*;&^<>—:A-Za-z]*[.!?]', text)

    pattern = '[.!?]'
    possible_ends_list = []
    possible_ends_indices = []

    p = re.compile(pattern)

    morph = pymorphy2.MorphAnalyzer()

    for m in p.finditer(text):
        index = m.start()
        character = m.group()
        possible_ends_list.append(character)
        possible_ends_indices.append(index)

    left_pseudowords_list = []
    right_pseudowords_list = []
    left_pseudowords_count = []
    right_pseudowords_count = []
    right_abb_list = []
    left_abb_list = []
    cright_list = []
    cleft_list = []
    wright_list = []
    cwright_list = []
    left_lengths = []
    left_caps = []
    right_caps = []

    for i in range(len(possible_sentences)):
        # load data
        cur_sentence = possible_sentences[i]
        if i + 1 != len(possible_sentences):
            next_sentence = possible_sentences[i + 1]
        else:
            next_sentence = ''
            
        # clean data
        cur_sentence = re.sub(' +', ' ', cur_sentence).strip('( )*')    
        next_sentence = re.sub(' +', ' ', next_sentence).strip('( )*')

        # extracting pseudowords from data
        left_pseudowords = re.split('[(\n)|( )]', cur_sentence)
        right_pseudowords = re.split('[(\n)|( )]', next_sentence)
        left_pseudoword = left_pseudowords[-1]
        right_pseudoword = right_pseudowords[0]
        left_abb = 0
        right_abb = 0
        cleft = ''
        cright = ''
        wright = list(filter(lambda v: match('[ЁёА-я0-9A-Za-z]+[ЁёА-я0-9 ,()+\-/"@#№$%_=*;&^<>—:A-Za-z]*', v), 
                                  right_pseudowords))        
        cwright = ''

        if len(wright) == 0:
            wright = '---'
        else:
            wright = wright[0].strip(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]*')

            """
            testing
            print(cur_sentence, left_pseudowords, left_pseudoword, len(left_pseudowords))
            print(next_sentence, right_pseudowords, right_pseudoword, len(right_pseudowords))
            print(len(possible_ends_list) == len(possible_sentences))
            """

        if len(left_pseudoword) > 1:
            left_pseudoword = left_pseudoword.strip(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]*')
            left_abb = find_abb(left_pseudoword, abr)
        elif left_pseudowords == ['']:
            left_pseudowords = []
        if len(left_pseudoword) == 0:
            left_pseudowords = []
            left_pseudoword = '---'

        if len(right_pseudoword) > 1:
            right_pseudoword = right_pseudowords[0].strip(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]*')
            right_abb = find_abb(right_pseudoword, abr)
        elif right_pseudowords == ['']:
            right_pseudowords = []
        if len(right_pseudoword) == 0:
            right_pseudowords = []
            right_pseudoword = '---'

        if wright != '---':
            is_cyrillic = re.findall('[А-я]', wright)
            is_latin = re.findall('[A-z]', wright)
            is_numeric = re.findall('[0-9]', wright)
            is_uppercase = re.findall('[А-Я]', wright[0])
            is_lowercase = re.findall('[а-я]', wright[0])
            is_first_num = re.findall('[0-9]', wright[0])
            is_first_punct = re.findall(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]', wright[0])

            if is_cyrillic:
                cwright += 'Cyrillic'
            if is_latin:
                if len(cwright) > 0:
                    cwright += ' + '
                cwright += 'Latin'
            if is_numeric:
                if len(cwright) > 0:
                    cwright += ' + '
                cwright += 'Numeric'
            if is_uppercase:
                if len(cwright) > 0:
                    cwright += ' + '
                cwright += 'Uppercase'
            if is_lowercase:
                if len(cwright) > 0:
                    cwright += ' + '
                cwright += 'Lowercase'

            if is_first_num:
                if len(cwright) > 0:
                    cwright += ' + '
                cwright += 'Numeric first'

            if is_first_punct:
                if len(cwright) > 0:
                    cwright += ' + '
                cwright += 'Punctuation first'

            if len(cwright) == 0:
                cwright = '---'
        else:
            cwright = '---'

        if len(right_pseudoword) > 0:
            is_cyrillic = re.findall('[А-я]', right_pseudoword)
            is_latin = re.findall('[A-z]', right_pseudoword)
            is_numeric = re.findall('[0-9]', right_pseudoword)
            is_uppercase = re.findall('[А-Я]', right_pseudoword[0])
            is_lowercase = re.findall('[а-я]', right_pseudoword[0])
            is_first_num = re.findall('[0-9]', right_pseudoword[0])
            is_first_punct = re.findall(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]', right_pseudoword[0])

            if is_cyrillic:
                cright += 'Cyrillic'
            if is_latin:
                if len(cright) > 0:
                    cright += ' + '
                cright += 'Latin'
            if is_numeric:
                if len(cright) > 0:
                    cright += ' + '
                cright += 'Numeric'
            if is_uppercase:
                if len(cright) > 0:
                    cright += ' + '
                cright += 'Uppercase'
            if is_lowercase:
                if len(cright) > 0:
                    cright += ' + '
                cright += 'Lowercase'

            if is_first_num:
                if len(cright) > 0:
                    cright += ' + '
                cright += 'Numeric first'

            if is_first_punct:
                if len(cright) > 0:
                    cright += ' + '
                cright += 'Punctuation first'

            if len(cright) == 0:
                cright = '---'

        else:
            cright = '---'

        if len(left_pseudoword) > 0:
            is_cyrillic = re.findall('[А-я]', left_pseudoword)
            is_latin = re.findall('[A-z]', left_pseudoword)
            is_numeric = re.findall('[0-9]', left_pseudoword)
            is_uppercase = re.findall('[А-Я]', left_pseudoword[0])
            is_lowercase = re.findall('[а-я]', left_pseudoword[0])
            is_first_num = re.findall('[0-9]', left_pseudoword[0])
            is_first_punct = re.findall(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]', left_pseudoword[0])

            if is_cyrillic:
                cleft += 'Cyrillic'
            if is_latin:
                if len(cleft) > 0:
                    cleft += ' + '
                cleft += 'Latin'
            if is_numeric:
                if len(cleft) > 0:
                    cleft += ' + '
                cleft += 'Numeric'
            if is_uppercase:
                if len(cleft) > 0:
                    cleft += ' + '
                cleft += 'Uppercase'
            if is_lowercase:
                if len(cleft) > 0:
                    cleft += ' + '
                cleft += 'Lowercase'

            if is_first_num:
                if len(cleft) > 0:
                    cleft += ' + '
                cleft += 'Numeric first'

            if is_first_punct:
                if len(cleft) > 0:
                    cleft += ' + '
                cleft += 'Punctuation first'

            if len(cleft) == 0:
                cleft = '---'

        else:
            cleft = '---'

            # saving pseudowords extracted data
        left_pseudowords_list.append(morph.parse(left_pseudoword)[0].normal_form)
        right_pseudowords_list.append(morph.parse(right_pseudoword)[0].normal_form)
        left_pseudowords_count.append(len(left_pseudowords))
        right_pseudowords_count.append(len(right_pseudowords))
        left_abb_list.append(left_abb)
        right_abb_list.append(right_abb)
        cleft_list.append(cleft)
        cright_list.append(cright)
        wright_list.append(morph.parse(wright)[0].normal_form)
        cwright_list.append(cwright)
        # my
        left_lengths.append(len(left_pseudoword))
        left_caps.append(len(re.findall('[А-Я]', left_pseudoword)) == len(left_pseudoword))
        right_caps.append(len(re.findall('[А-Я]', right_pseudoword)) == len(right_pseudoword))

        data_for_df = dict()

        data_for_df['sign'] = possible_ends_list
        data_for_df['sign index'] = possible_ends_indices
        data_for_df['right'] = right_pseudowords_list
        data_for_df['left'] = left_pseudowords_list
        data_for_df['dleft'] = left_pseudowords_count
        data_for_df['dright'] = right_pseudowords_count
        data_for_df['abbleft'] = left_abb_list
        data_for_df['abbright'] = right_abb_list
        data_for_df['cright'] = cright_list
        data_for_df['cleft'] = cleft_list
        data_for_df['wright'] = wright_list
        data_for_df['cwright'] = cwright_list
        data_for_df['left_caps'] = left_caps
        data_for_df['right_caps'] = right_caps
        data_for_df['end?'] = answers

    possible_ends_df = pd.DataFrame(data_for_df)
    
    return possible_ends_df

def test():
    with open('raw_text/Постановление ФАС Уральского округа от 01_06_2006 N Ф09-3415_rzm', "rb") as fp:
        anss1 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 01_06_2006 N Ф09-3415.txt',
             encoding='cp1251', errors='ignore') as fp:
        text1 = fp.read()
        # print(text)
    fp.close()

    # tablee1 = make_table(text1, anss1)

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-2600_rzm', "rb") as fp:
        anss2 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-2600.txt',
             encoding='cp1251', errors='ignore') as fp:
        text2 = fp.read()
        # print(text)
    fp.close()

    # tablee2 = make_table(text2, anss2)

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-4528_rzm', "rb") as fp:
        anss3 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-4528.txt',
             encoding='cp1251', errors='ignore') as fp:
        text3 = fp.read()
        # print(text)
    fp.close()

    # tablee3 = make_table(text3, anss3)

    with open('raw_text/Постановление ФАС Уральского округа от 20_12_2010 N Ф09-1066_rzm', "rb") as fp:
        anss4 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 20_12_2010 N Ф09-1066.txt',
             encoding='cp1251', errors='ignore') as fp:
        text4 = fp.read()
        # print(text)
    fp.close()

    # tablee4 = make_table(text4, anss4)

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-4409_rzm', "rb") as fp:
        anss5 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-4409.txt',
             encoding='cp1251', errors='ignore') as fp:
        text5 = fp.read()
        # print(text)
    fp.close()

    # tablee5 = make_table(text5, anss5)

    with open('raw_text/Постановление ФАС Уральского округа от 15_12_2011 N Ф09-8320_rzm', "rb") as fp:
        anss6 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 15_12_2011 N Ф09-8320.txt',
             encoding='cp1251', errors='ignore') as fp:
        text6 = fp.read()
        # print(text)
    fp.close()

    # tablee6 = make_table(text6, anss6)

    with open('raw_text/Постановление ФАС Уральского округа от 30_05_2006 N Ф09-4438_rzm', "rb") as fp:
        anss7 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 30_05_2006 N Ф09-4438.txt',
             encoding='cp1251', errors='ignore') as fp:
        text7 = fp.read()
        # print(text)
    fp.close()

    # tablee7 = make_table(text7, anss7)

    with open('raw_text/Постановление ФАС Уральского округа от 29_01_2002 N Ф09-92 0_rzm', "rb") as fp:
        anss8 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 29_01_2002 N Ф09-92 0.txt',
             encoding='cp1251', errors='ignore') as fp:
        text8 = fp.read()
        # print(text)
    fp.close()

    # tablee8 = make_table(text8, anss8)

    with open('raw_text/Постановление ФАС Уральского округа от 27_07_2012 N Ф09-6215_rzm', "rb") as fp:
        anss9 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 27_07_2012 N Ф09-6215.txt',
             encoding='cp1251', errors='ignore') as fp:
        text9 = fp.read()
        # print(text)
    fp.close()

    # tablee9 = make_table(text9, anss9)

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-4435_rzm', "rb") as fp:
        anss10 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-4435.txt',
             encoding='cp1251', errors='ignore') as fp:
        text10 = fp.read()
        # print(text)
    fp.close()

    # tablee10 = make_table(text10, anss10); плохо сделал именно этот текст

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-4559_rzm', "rb") as fp:
        anss11 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-4559.txt',
             encoding='cp1251', errors='ignore') as fp:
        text11 = fp.read()
        # print(text)
    fp.close()

    # tablee11

    with open('raw_text/Постановление ФАС Уральского округа от 20_12_2010 N Ф09-9906_rzm', "rb") as fp:
        anss12 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 20_12_2010 N Ф09-9906.txt',
             encoding='cp1251', errors='ignore') as fp:
        text12 = fp.read()
        # print(text)
    fp.close()

    # tablee12

    with open('raw_text/Постановление ФАС Уральского округа от 30_05_2006 N Ф09-4336_rzm', "rb") as fp:
        anss13 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 30_05_2006 N Ф09-4336.txt',
             encoding='cp1251', errors='ignore') as fp:
        text13 = fp.read()
        # print(text)
    fp.close()

    # tablee13

    with open('raw_text/Постановление ФАС Уральского округа от 05_06_2006 N Ф09-205_rzm', "rb") as fp:
        anss14 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 05_06_2006 N Ф09-205.txt',
             encoding='cp1251', errors='ignore') as fp:
        text14 = fp.read()
        # print(text)
    fp.close()

    # tablee14

    with open('raw_text/Постановление ФАС Уральского округа от 30_05_2006 N Ф09-4131_rzm', "rb") as fp:
        anss15 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 30_05_2006 N Ф09-4131.txt',
             encoding='cp1251', errors='ignore') as fp:
        text15 = fp.read()
        # print(text)
    fp.close()

    # tablee15

    with open('raw_text/Постановление ФАС Уральского округа от 06_02_2007 N Ф09-4881_rzm', "rb") as fp:
        anss16 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 06_02_2007 N Ф09-4881.txt',
             encoding='cp1251', errors='ignore') as fp:
        text16 = fp.read()
        # print(text)
    fp.close()

    # tablee16

    with open('raw_text/Постановление ФАС Уральского округа от 30_07_2012 N Ф09-1027_rzm', "rb") as fp:
        anss17 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 30_07_2012 N Ф09-1027.txt',
             encoding='cp1251', errors='ignore') as fp:
        text17 = fp.read()
        # print(text)
    fp.close()

    # tablee17

    with open('raw_text/Постановление ФАС Уральского округа от 01_06_2006 N Ф09-4469_rzm', "rb") as fp:
        anss18 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 01_06_2006 N Ф09-4469.txt',
             encoding='cp1251', errors='ignore') as fp:
        text18 = fp.read()
        # print(text)
    fp.close()

    # tablee18

    with open('raw_text/Постановление ФАС Уральского округа от 20_12_2010 N Ф09-1060_rzm', "rb") as fp:
        anss19 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 20_12_2010 N Ф09-1060.txt',
             encoding='cp1251', errors='ignore') as fp:
        text19 = fp.read()
        # print(text)
    fp.close()

    # tablee19

    with open('raw_text/Постановление ФАС Уральского округа от 01_06_2006 N Ф09-4398_rzm', "rb") as fp:
        anss20 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 01_06_2006 N Ф09-4398.txt',
             encoding='cp1251', errors='ignore') as fp:
        text20 = fp.read()
        # print(text)
    fp.close()

    # tablee20

    def ohe_tr(onehotencoder, data, categorical_cols, not_fit):
        if not_fit:
            transformed_data = onehotencoder.fit_transform(data[categorical_cols])
        else:
            transformed_data = onehotencoder.transform(data[categorical_cols])
        encoded_data = pd.DataFrame(transformed_data, index=data.index)
        other_data = data.drop(columns=categorical_cols)
        data_out = pd.concat([encoded_data, other_data], axis=1)
        return data_out

    def train_test(n):
        texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10,
                text11, text12, text13, text14, text15, text16, text17, text18, text19,
                text20]
        anss = [anss1, anss2, anss3, anss4, anss5, anss6, anss7, anss8, anss9, anss10,
               anss11, anss12, anss13, anss14, anss15, anss16, anss17, anss18, anss19,
               anss20]

        bigtext = ''
        bigans = []

        for text in texts[:n] + texts[n + 1:]:
            bigtext += text

        for ans in anss[:n] + anss[n + 1:]:
            bigans += ans

        table_train = make_table(bigtext, bigans)
        table_test = make_table(texts[n], anss[n])

        X_train = table_train.drop(['sign index', 'end?'], axis=1)
        y_train = table_train['end?']

        categorical = ['sign', 'wright', 'right', 'left', 'abbleft', 'abbright', 'cright', 'cleft', 'cwright', 'left_caps', 'right_caps']

        X_train_trans = ohe_tr(enc, X_train, categorical, True)

        clf = GradientBoostingClassifier(loss='deviance', max_depth=3)
        clf.fit(X_train_trans, y_train)

        X_test = table_test.drop(['sign index', 'end?'], axis=1)
        y_test = table_test['end?']

        X_test_trans = ohe_tr(enc, X_test, categorical, False)
        y_pred = clf.predict(X_test_trans)
        
        return f1_score(y_test, y_pred)

    cnt = 20
    for k in range(0, cnt):
        print(k + 1, '/', cnt, train_test(k))

4. Функция для `fit` OneHotEncoder'а

In [ ]:
texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10,
        text11, text12, text13, text14, text15, text16, text17, text18, text19,
        text20]
anss = [anss1, anss2, anss3, anss4, anss5, anss6, anss7, anss8, anss9, anss10,
        anss11, anss12, anss13, anss14, anss15, anss16, anss17, anss18, anss19,
        anss20]

bigtext = ''
bigans = []

for text in texts:
    bigtext += text

for ans in anss:
    bigans += ans

table_train = make_table(bigtext)
X_train = table_train.drop(['sign index'], axis=1)
categorical = ['sign', 'wright', 'right', 'left', 'abbleft', 'abbright', 'cright', 'cleft', 'cwright', 'left_caps', 'right_caps']

enc = OneHotEncoder(handle_unknown='ignore', sparse=False, categories='auto')
enc.fit(X_train[categorical])

with open('enc.pkl','wb') as f:
    pickle.dump(enc,f)
f.close()

5. Функция для обучения бустинга:

In [ ]:
texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10,
        text11, text12, text13, text14, text15, text16, text17, text18, text19,
        text20]
anss = [anss1, anss2, anss3, anss4, anss5, anss6, anss7, anss8, anss9, anss10,
        anss11, anss12, anss13, anss14, anss15, anss16, anss17, anss18, anss19,
        anss20]

bigtext = ''
bigans = []

for text in texts:
    bigtext += text

for ans in anss:
    bigans += ans

table_train = make_table(bigtext, bigans)
table_test = make_table(text1, anss1)

X_train = table_train.drop(['sign index', 'end?'], axis=1)
y_train = table_train['end?']
    
categorical = ['sign', 'wright', 'right', 'left', 'abbleft', 'abbright', 'cright', 'cleft', 'cwright', 'left_caps', 'right_caps']
enc = OneHotEncoder(handle_unknown='ignore', sparse=False, categories='auto')
X_train_trans = ohe_tr(enc, X_train, categorical, True)

clf = GradientBoostingClassifier(loss='deviance', max_depth=3)
clf.fit(X_train_trans, y_train)

with open('model.pkl','wb') as f:
    pickle.dump(clf,f)
f.close()

6. Функция для получения признаков для примеров 1–12 из статьи:

In [ ]:
texts = ['Конечно, в рамках газетной статьи невозможно сделать обзор сотни докладов, поэтому мы рекомендуем посетить Интернет-страницу конференции http://www.ict.nsc.ru/ws/mol2000/, на которой размещены программа мероприятия и тезисы докладов.',
        'В 11.45 дали слово Кудрину , но он всё не шёл.',
        'выполнение 12 тепловозам усиленного ТР-1 с применением средств диагностики вместо ТР-2 дало экономический эффект 221,8 тыс. руб.',
        """Параметры системы питания :
        линейное напряжение на проводах 81 ... 83 , В ......... 220
        фазное напряжение на проводах 81 ... 83 по отношению
        к проводу 84 ( нулю ), В ................................................ 127
        частота переменного тока, Гц .................................... 50
        выпрямленное напряжение на проводах, В
        15--30............................................................... 110
        44--30................................................................. 50""",
        'Режиссёр Михаил. Бычков поставил в Таллине притчу о любви к невозможному и о презрении к реальности .',
        'Дело в том, что по всяким планам «пятилеток» и заданиям ЦК советский военный комплекс создавал ядерное оружие с запасом на пять и более(!) ядерных войн.',
        'Пролетели "Тише!" Виктора Косаковского и "Фрески" Александра Гутмана.',
        'Не в лесе и не в медицине дело...',
        'Только вот ради чего ?!',
        'В связи с этим первый интервал пробегов был принят равным 350...700 тыс. км (середина интервала - 525 тыс. км), второй интервал -- 700...1050 тыс. км (середина интервала - 875 тыс. км) и третий интервал 1050...1400 тыс. км (середина интервала -- 1225 тыс. км).',
        'В случае нарушений выносится письменное предупреждение: «В вашей деятельности допускаются вот такие недочёты ...»',
        ]
answers = [[0, 0, 0, 1],
          [0, 1],
          [0, 1],
          [0] * ("""Параметры системы питания :
                линейное напряжение на проводах 81 ... 83 , В ......... 220
                фазное напряжение на проводах 81 ... 83 по отношению
                к проводу 84 ( нулю ), В ................................................ 127
                частота переменного тока, Гц .................................... 50
                выпрямленное напряжение на проводах, В
                15--30............................................................... 110
                44--30................................................................. 50""".count('.') - 1) + [1],
          [0, 1],
          [0, 1],
          [0, 1],
          [0, 0, 1],
          [0, 1],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
          [0, 0, 1]]

trytexts = texts[0]
tryanswers = answers[0]

for trytext in texts[1:]:
    trytexts += '\n'
    trytexts += trytext
    
for tryanswer in answers[1:]:
    tryanswers += tryanswer
    
table = make_table(trytexts, tryanswers)

7. Функция для тюнинга гиперпараметров бустинга:

In [ ]:
parameters = {
    "loss":["deviance", "exponential"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }

def ohe_tr(onehotencoder, data, categorical_cols, not_fit):
    if not_fit:
        transformed_data = onehotencoder.fit_transform(data[categorical_cols])
    else:
        transformed_data = onehotencoder.transform(data[categorical_cols])
    encoded_data = pd.DataFrame(transformed_data, index=data.index)
    other_data = data.drop(columns=categorical_cols)
    data_out = pd.concat([encoded_data, other_data], axis=1)
    return data_out

def train_test(n):
    texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10, trytexts,
            text11, text12, text13, text14]
    anss = [anss1, anss2, anss3, anss4, anss5, anss6, anss7, anss8, anss9, anss10, tryanswers,
           anss11, anss12, anss13, anss14]

    bigtext = ''
    bigans = []

    for text in texts[:n] + texts[n + 1:]:
        bigtext += text

    for ans in anss[:n] + anss[n + 1:]:
        bigans += ans

    table_train = make_table(bigtext, bigans)
    table_test = make_table(texts[n], anss[n])

    X_train = table_train.drop(['sign index', 'end?'], axis=1)
    y_train = table_train['end?']
    
    categorical = ['sign', 'wright', 'right', 'left', 'abbleft', 'abbright', 'cright', 
                   'cleft', 'cwright', 'left_caps', 'right_caps']
    enc = OneHotEncoder(handle_unknown='ignore', sparse=False, categories='auto')
   
    X_train_trans = ohe_tr(enc, X_train, categorical, True)
    X_test = table_test.drop(['sign index', 'end?'], axis=1)
    y_test = table_test['end?']
    X_test_trans = ohe_tr(enc, X_test, categorical, False)

    clf = GridSearchCV(
            GradientBoostingClassifier(), parameters, scoring=f1, n_jobs= -1, cv = 3
        )
    clf.fit(X_train_trans, y_train)
    clf.best_params_
    
    return clf.best_params_

cnt = 14
for k in range(0, cnt + 1):
    print(k+1, '/', cnt + 1, train_test(k))

8. Программа, разбивающая текст на предложения (итог работы):

In [40]:
def start():
    # !pip install opencorpora-tools
    # !pip install -U pymorphy2-dicts-ru

    import pandas as pd
    import re
    from sklearn import svm
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.metrics import f1_score
    import opencorpora
    import os
    import pickle
    from re import match
    import pymorphy2
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import f1_score, make_scorer
    import numpy as np

    with open("abbrevs", "rb") as fp:
        abr = pickle.load(fp)
    fp.close()

    def find_abb(word, abbreviatures_list):
        abbreviatures = dict()
        for sent in abbreviatures_list:
            srch = re.findall('(^|\s)' + word + '[.]+[ ]+[А-Я]+[А-я]*', sent)
            if srch:
                return 1
        return 0

    def make_table(text):
        possible_sentences = re.findall('[ЁёА-я0-9 ,()+\-/"@#№$%_=*;&^<>—:A-Za-z]*[.!?]', text)

        pattern = '[.!?]'
        possible_ends_list = []
        possible_ends_indices = []

        p = re.compile(pattern)

        morph = pymorphy2.MorphAnalyzer()

        for m in p.finditer(text):
            index = m.start()
            character = m.group()
            possible_ends_list.append(character)
            possible_ends_indices.append(index)

        left_pseudowords_list = []
        right_pseudowords_list = []
        left_pseudowords_count = []
        right_pseudowords_count = []
        right_abb_list = []
        left_abb_list = []
        cright_list = []
        cleft_list = []
        wright_list = []
        cwright_list = []
        # my
        left_lengths = []
        left_caps = []
        right_caps = []

        for i in range(len(possible_sentences)):
            # load data
            cur_sentence = possible_sentences[i]
            if i + 1 != len(possible_sentences):
                next_sentence = possible_sentences[i + 1]
            else:
                next_sentence = ''

            # clean data
            cur_sentence = re.sub(' +', ' ', cur_sentence).strip('( )*')    
            next_sentence = re.sub(' +', ' ', next_sentence).strip('( )*')

            # extracting pseudowords from data
            left_pseudowords = re.split('[(\n)|( )]', cur_sentence)
            right_pseudowords = re.split('[(\n)|( )]', next_sentence)
            left_pseudoword = left_pseudowords[-1]
            right_pseudoword = right_pseudowords[0]
            left_abb = 0
            right_abb = 0
            cleft = ''
            cright = ''
            wright = list(filter(lambda v: match('[ЁёА-я0-9A-Za-z]+[ЁёА-я0-9 ,()+\-/"@#№$%_=*;&^<>—:A-Za-z]*', v), 
                                  right_pseudowords))        
            cwright = ''

            if len(wright) == 0:
                wright = '---'
            else:
                wright = wright[0].strip(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]*')

            """
            testing
            print(cur_sentence, left_pseudowords, left_pseudoword, len(left_pseudowords))
            print(next_sentence, right_pseudowords, right_pseudoword, len(right_pseudowords))
            print(len(possible_ends_list) == len(possible_sentences))
            """

            if len(left_pseudoword) > 1:
                left_pseudoword = left_pseudoword.strip(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]*')
                left_abb = find_abb(left_pseudoword, abr)
            elif left_pseudowords == ['']:
                left_pseudowords = []
            if len(left_pseudoword) == 0:
                left_pseudowords = []
                left_pseudoword = '---'

            if len(right_pseudoword) > 1:
                right_pseudoword = right_pseudowords[0].strip(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]*')
                right_abb = find_abb(right_pseudoword, abr)
            elif right_pseudowords == ['']:
                right_pseudowords = []
            if len(right_pseudoword) == 0:
                right_pseudowords = []
                right_pseudoword = '---'

            if wright != '---':
                is_cyrillic = re.findall('[А-я]', wright)
                is_latin = re.findall('[A-z]', wright)
                is_numeric = re.findall('[0-9]', wright)
                is_uppercase = re.findall('[А-Я]', wright[0])
                is_lowercase = re.findall('[а-я]', wright[0])
                is_first_num = re.findall('[0-9]', wright[0])
                is_first_punct = re.findall(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]', wright[0])

                if is_cyrillic:
                    cwright += 'Cyrillic'
                if is_latin:
                    if len(cwright) > 0:
                        cwright += ' + '
                    cwright += 'Latin'
                if is_numeric:
                    if len(cwright) > 0:
                        cwright += ' + '
                    cwright += 'Numeric'
                if is_uppercase:
                    if len(cwright) > 0:
                        cwright += ' + '
                    cwright += 'Uppercase'
                if is_lowercase:
                    if len(cwright) > 0:
                        cwright += ' + '
                    cwright += 'Lowercase'

                if is_first_num:
                    if len(cwright) > 0:
                        cwright += ' + '
                    cwright += 'Numeric first'

                if is_first_punct:
                    if len(cwright) > 0:
                        cwright += ' + '
                    cwright += 'Punctuation first'

                if len(cwright) == 0:
                    cwright = '---'
            else:
                cwright = '---'

            if len(right_pseudoword) > 0:
                is_cyrillic = re.findall('[А-я]', right_pseudoword)
                is_latin = re.findall('[A-z]', right_pseudoword)
                is_numeric = re.findall('[0-9]', right_pseudoword)
                is_uppercase = re.findall('[А-Я]', right_pseudoword[0])
                is_lowercase = re.findall('[а-я]', right_pseudoword[0])
                is_first_num = re.findall('[0-9]', right_pseudoword[0])
                is_first_punct = re.findall(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]', right_pseudoword[0])

                if is_cyrillic:
                    cright += 'Cyrillic'
                if is_latin:
                    if len(cright) > 0:
                        cright += ' + '
                    cright += 'Latin'
                if is_numeric:
                    if len(cright) > 0:
                        cright += ' + '
                    cright += 'Numeric'
                if is_uppercase:
                    if len(cright) > 0:
                        cright += ' + '
                    cright += 'Uppercase'
                if is_lowercase:
                    if len(cright) > 0:
                        cright += ' + '
                    cright += 'Lowercase'

                if is_first_num:
                    if len(cright) > 0:
                        cright += ' + '
                    cright += 'Numeric first'

                if is_first_punct:
                    if len(cright) > 0:
                        cright += ' + '
                    cright += 'Punctuation first'

                if len(cright) == 0:
                    cright = '---'

            else:
                cright = '---'

            if len(left_pseudoword) > 0:
                is_cyrillic = re.findall('[А-я]', left_pseudoword)
                is_latin = re.findall('[A-z]', left_pseudoword)
                is_numeric = re.findall('[0-9]', left_pseudoword)
                is_uppercase = re.findall('[А-Я]', left_pseudoword[0])
                is_lowercase = re.findall('[а-я]', left_pseudoword[0])
                is_first_num = re.findall('[0-9]', left_pseudoword[0])
                is_first_punct = re.findall(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]', left_pseudoword[0])

                if is_cyrillic:
                    cleft += 'Cyrillic'
                if is_latin:
                    if len(cleft) > 0:
                        cleft += ' + '
                    cleft += 'Latin'
                if is_numeric:
                    if len(cleft) > 0:
                        cleft += ' + '
                    cleft += 'Numeric'
                if is_uppercase:
                    if len(cleft) > 0:
                        cleft += ' + '
                    cleft += 'Uppercase'
                if is_lowercase:
                    if len(cleft) > 0:
                        cleft += ' + '
                    cleft += 'Lowercase'

                if is_first_num:
                    if len(cleft) > 0:
                        cleft += ' + '
                    cleft += 'Numeric first'

                if is_first_punct:
                    if len(cleft) > 0:
                        cleft += ' + '
                    cleft += 'Punctuation first'

                if len(cleft) == 0:
                    cleft = '---'

            else:
                cleft = '---'

            # saving pseudowords extracted data
            left_pseudowords_list.append(morph.parse(left_pseudoword)[0].normal_form)
            right_pseudowords_list.append(morph.parse(right_pseudoword)[0].normal_form)
            left_pseudowords_count.append(len(left_pseudowords))
            right_pseudowords_count.append(len(right_pseudowords))
            left_abb_list.append(left_abb)
            right_abb_list.append(right_abb)
            cleft_list.append(cleft)
            cright_list.append(cright)
            wright_list.append(morph.parse(wright)[0].normal_form)
            cwright_list.append(cwright)
            # my
            left_lengths.append(len(left_pseudoword))
            left_caps.append(len(re.findall('[А-Я]', left_pseudoword)) == len(left_pseudoword))
            right_caps.append(len(re.findall('[А-Я]', right_pseudoword)) == len(right_pseudoword))

        data_for_df = dict()

        data_for_df['sign'] = possible_ends_list
        data_for_df['sign index'] = possible_ends_indices
        data_for_df['right'] = right_pseudowords_list
        data_for_df['left'] = left_pseudowords_list
        data_for_df['dleft'] = left_pseudowords_count
        data_for_df['dright'] = right_pseudowords_count
        data_for_df['abbleft'] = left_abb_list
        data_for_df['abbright'] = right_abb_list
        data_for_df['cright'] = cright_list
        data_for_df['cleft'] = cleft_list
        data_for_df['wright'] = wright_list
        data_for_df['cwright'] = cwright_list
        data_for_df['left_caps'] = left_caps
        data_for_df['right_caps'] = right_caps

        possible_ends_df = pd.DataFrame(data_for_df)

        return possible_ends_df

    def ohe_tr(onehotencoder, data, categorical_cols, not_fit):
        if not_fit:
            transformed_data = onehotencoder.fit_transform(data[categorical_cols])
        else:
            transformed_data = onehotencoder.transform(data[categorical_cols])
        encoded_data = pd.DataFrame(transformed_data, index=data.index)
        other_data = data.drop(columns=categorical_cols)
        data_out = pd.concat([encoded_data, other_data], axis=1)
        return data_out

    with open('model.pkl', 'rb') as f:
        clf = pickle.load(f)
    f.close()

    print('Введите название текстового файла с постановлением ФАС Уральского округа')

    filename = input()

    with open(filename, encoding='cp1251', errors='ignore') as f:
        text = f.read()
    f.close()

    with open('enc.pkl', 'rb') as fp:
        enc = pickle.load(fp)
    fp.close()

    table_test = make_table(text)

    X_test = table_test.drop(['sign index'], axis=1)

    X_test_trans = ohe_tr(enc, X_test, categorical, False)
    y_pred = clf.predict(X_test_trans)

    answer_sentences = re.findall('[ЁёА-я0-9 ,()+\-/"@#№$%_=*;&^<>—:A-Za-z\n\t]*[.!?]', text)
    answer = ''
    for i in range(len(answer_sentences)):
        if y_pred[i] == 1:
            answer += answer_sentences[i].strip('( )*') + '\n'
        else:
            answer += answer_sentences[i].strip('( )*')

    print(answer)

9. Функция для тестирования прочих моделей:

In [47]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

def find_abb(word, abbreviatures_list):
    abbreviatures = dict()
    for sent in abbreviatures_list:
        srch = re.findall('(^|\s)' + word + '[.]+[ ]+[А-Я]+[А-я]*', sent)
        if srch:
            return 1
    return 0

def make_table(text, answers):
    possible_sentences = re.findall('[ЁёА-я0-9 ,()+\-/"@#№$%_=*;&^<>—:A-Za-z]*[.!?]', text)

    pattern = '[.!?]'
    possible_ends_list = []
    possible_ends_indices = []

    p = re.compile(pattern)

    morph = pymorphy2.MorphAnalyzer()

    for m in p.finditer(text):
        index = m.start()
        character = m.group()
        possible_ends_list.append(character)
        possible_ends_indices.append(index)

    left_pseudowords_list = []
    right_pseudowords_list = []
    left_pseudowords_count = []
    right_pseudowords_count = []
    right_abb_list = []
    left_abb_list = []
    cright_list = []
    cleft_list = []
    wright_list = []
    cwright_list = []
    left_lengths = []
    left_caps = []
    right_caps = []

    for i in range(len(possible_sentences)):
        # load data
        cur_sentence = possible_sentences[i]
        if i + 1 != len(possible_sentences):
            next_sentence = possible_sentences[i + 1]
        else:
            next_sentence = ''
            
        # clean data
        cur_sentence = re.sub(' +', ' ', cur_sentence).strip('( )*')    
        next_sentence = re.sub(' +', ' ', next_sentence).strip('( )*')

        # extracting pseudowords from data
        left_pseudowords = re.split('[(\n)|( )]', cur_sentence)
        right_pseudowords = re.split('[(\n)|( )]', next_sentence)
        left_pseudoword = left_pseudowords[-1]
        right_pseudoword = right_pseudowords[0]
        left_abb = 0
        right_abb = 0
        cleft = ''
        cright = ''
        wright = list(filter(lambda v: match('[ЁёА-я0-9A-Za-z]+[ЁёА-я0-9 ,()+\-/"@#№$%_=*;&^<>—:A-Za-z]*', v), 
                                  right_pseudowords))        
        cwright = ''

        if len(wright) == 0:
            wright = '---'
        else:
            wright = wright[0].strip(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]*')

            """
            testing
            print(cur_sentence, left_pseudowords, left_pseudoword, len(left_pseudowords))
            print(next_sentence, right_pseudowords, right_pseudoword, len(right_pseudowords))
            print(len(possible_ends_list) == len(possible_sentences))
            """

        if len(left_pseudoword) > 1:
            left_pseudoword = left_pseudoword.strip(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]*')
            left_abb = find_abb(left_pseudoword, abr)
        elif left_pseudowords == ['']:
            left_pseudowords = []
        if len(left_pseudoword) == 0:
            left_pseudowords = []
            left_pseudoword = '---'

        if len(right_pseudoword) > 1:
            right_pseudoword = right_pseudowords[0].strip(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]*')
            right_abb = find_abb(right_pseudoword, abr)
        elif right_pseudowords == ['']:
            right_pseudowords = []
        if len(right_pseudoword) == 0:
            right_pseudowords = []
            right_pseudoword = '---'

        if wright != '---':
            is_cyrillic = re.findall('[А-я]', wright)
            is_latin = re.findall('[A-z]', wright)
            is_numeric = re.findall('[0-9]', wright)
            is_uppercase = re.findall('[А-Я]', wright[0])
            is_lowercase = re.findall('[а-я]', wright[0])
            is_first_num = re.findall('[0-9]', wright[0])
            is_first_punct = re.findall(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]', wright[0])

            if is_cyrillic:
                cwright += 'Cyrillic'
            if is_latin:
                if len(cwright) > 0:
                    cwright += ' + '
                cwright += 'Latin'
            if is_numeric:
                if len(cwright) > 0:
                    cwright += ' + '
                cwright += 'Numeric'
            if is_uppercase:
                if len(cwright) > 0:
                    cwright += ' + '
                cwright += 'Uppercase'
            if is_lowercase:
                if len(cwright) > 0:
                    cwright += ' + '
                cwright += 'Lowercase'

            if is_first_num:
                if len(cwright) > 0:
                    cwright += ' + '
                cwright += 'Numeric first'

            if is_first_punct:
                if len(cwright) > 0:
                    cwright += ' + '
                cwright += 'Punctuation first'

            if len(cwright) == 0:
                cwright = '---'
        else:
            cwright = '---'

        if len(right_pseudoword) > 0:
            is_cyrillic = re.findall('[А-я]', right_pseudoword)
            is_latin = re.findall('[A-z]', right_pseudoword)
            is_numeric = re.findall('[0-9]', right_pseudoword)
            is_uppercase = re.findall('[А-Я]', right_pseudoword[0])
            is_lowercase = re.findall('[а-я]', right_pseudoword[0])
            is_first_num = re.findall('[0-9]', right_pseudoword[0])
            is_first_punct = re.findall(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]', right_pseudoword[0])

            if is_cyrillic:
                cright += 'Cyrillic'
            if is_latin:
                if len(cright) > 0:
                    cright += ' + '
                cright += 'Latin'
            if is_numeric:
                if len(cright) > 0:
                    cright += ' + '
                cright += 'Numeric'
            if is_uppercase:
                if len(cright) > 0:
                    cright += ' + '
                cright += 'Uppercase'
            if is_lowercase:
                if len(cright) > 0:
                    cright += ' + '
                cright += 'Lowercase'

            if is_first_num:
                if len(cright) > 0:
                    cright += ' + '
                cright += 'Numeric first'

            if is_first_punct:
                if len(cright) > 0:
                    cright += ' + '
                cright += 'Punctuation first'

            if len(cright) == 0:
                cright = '---'

        else:
            cright = '---'

        if len(left_pseudoword) > 0:
            is_cyrillic = re.findall('[А-я]', left_pseudoword)
            is_latin = re.findall('[A-z]', left_pseudoword)
            is_numeric = re.findall('[0-9]', left_pseudoword)
            is_uppercase = re.findall('[А-Я]', left_pseudoword[0])
            is_lowercase = re.findall('[а-я]', left_pseudoword[0])
            is_first_num = re.findall('[0-9]', left_pseudoword[0])
            is_first_punct = re.findall(',()+\-/"@#№$%_=*;&^<>—:A-Za-z.!?]', left_pseudoword[0])

            if is_cyrillic:
                cleft += 'Cyrillic'
            if is_latin:
                if len(cleft) > 0:
                    cleft += ' + '
                cleft += 'Latin'
            if is_numeric:
                if len(cleft) > 0:
                    cleft += ' + '
                cleft += 'Numeric'
            if is_uppercase:
                if len(cleft) > 0:
                    cleft += ' + '
                cleft += 'Uppercase'
            if is_lowercase:
                if len(cleft) > 0:
                    cleft += ' + '
                cleft += 'Lowercase'

            if is_first_num:
                if len(cleft) > 0:
                    cleft += ' + '
                cleft += 'Numeric first'

            if is_first_punct:
                if len(cleft) > 0:
                    cleft += ' + '
                cleft += 'Punctuation first'

            if len(cleft) == 0:
                cleft = '---'

        else:
            cleft = '---'

            # saving pseudowords extracted data
        left_pseudowords_list.append(morph.parse(left_pseudoword)[0].normal_form)
        right_pseudowords_list.append(morph.parse(right_pseudoword)[0].normal_form)
        left_pseudowords_count.append(len(left_pseudowords))
        right_pseudowords_count.append(len(right_pseudowords))
        left_abb_list.append(left_abb)
        right_abb_list.append(right_abb)
        cleft_list.append(cleft)
        cright_list.append(cright)
        wright_list.append(morph.parse(wright)[0].normal_form)
        cwright_list.append(cwright)
        # my
        left_lengths.append(len(left_pseudoword))
        left_caps.append(len(re.findall('[А-Я]', left_pseudoword)) == len(left_pseudoword))
        right_caps.append(len(re.findall('[А-Я]', right_pseudoword)) == len(right_pseudoword))

        data_for_df = dict()

        data_for_df['sign'] = possible_ends_list
        data_for_df['sign index'] = possible_ends_indices
        data_for_df['right'] = right_pseudowords_list
        data_for_df['left'] = left_pseudowords_list
        data_for_df['dleft'] = left_pseudowords_count
        data_for_df['dright'] = right_pseudowords_count
        data_for_df['abbleft'] = left_abb_list
        data_for_df['abbright'] = right_abb_list
        data_for_df['cright'] = cright_list
        data_for_df['cleft'] = cleft_list
        data_for_df['wright'] = wright_list
        data_for_df['cwright'] = cwright_list
        data_for_df['left_caps'] = left_caps
        data_for_df['right_caps'] = right_caps
        data_for_df['end?'] = answers

    possible_ends_df = pd.DataFrame(data_for_df)
    
    return possible_ends_df

def test_other():
    with open('raw_text/Постановление ФАС Уральского округа от 01_06_2006 N Ф09-3415_rzm', "rb") as fp:
        anss1 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 01_06_2006 N Ф09-3415.txt',
             encoding='cp1251', errors='ignore') as fp:
        text1 = fp.read()
        # print(text)
    fp.close()

    # tablee1 = make_table(text1, anss1)

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-2600_rzm', "rb") as fp:
        anss2 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-2600.txt',
             encoding='cp1251', errors='ignore') as fp:
        text2 = fp.read()
        # print(text)
    fp.close()

    # tablee2 = make_table(text2, anss2)

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-4528_rzm', "rb") as fp:
        anss3 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-4528.txt',
             encoding='cp1251', errors='ignore') as fp:
        text3 = fp.read()
        # print(text)
    fp.close()

    # tablee3 = make_table(text3, anss3)

    with open('raw_text/Постановление ФАС Уральского округа от 20_12_2010 N Ф09-1066_rzm', "rb") as fp:
        anss4 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 20_12_2010 N Ф09-1066.txt',
             encoding='cp1251', errors='ignore') as fp:
        text4 = fp.read()
        # print(text)
    fp.close()

    # tablee4 = make_table(text4, anss4)

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-4409_rzm', "rb") as fp:
        anss5 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-4409.txt',
             encoding='cp1251', errors='ignore') as fp:
        text5 = fp.read()
        # print(text)
    fp.close()

    # tablee5 = make_table(text5, anss5)

    with open('raw_text/Постановление ФАС Уральского округа от 15_12_2011 N Ф09-8320_rzm', "rb") as fp:
        anss6 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 15_12_2011 N Ф09-8320.txt',
             encoding='cp1251', errors='ignore') as fp:
        text6 = fp.read()
        # print(text)
    fp.close()

    # tablee6 = make_table(text6, anss6)

    with open('raw_text/Постановление ФАС Уральского округа от 30_05_2006 N Ф09-4438_rzm', "rb") as fp:
        anss7 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 30_05_2006 N Ф09-4438.txt',
             encoding='cp1251', errors='ignore') as fp:
        text7 = fp.read()
        # print(text)
    fp.close()

    # tablee7 = make_table(text7, anss7)

    with open('raw_text/Постановление ФАС Уральского округа от 29_01_2002 N Ф09-92 0_rzm', "rb") as fp:
        anss8 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 29_01_2002 N Ф09-92 0.txt',
             encoding='cp1251', errors='ignore') as fp:
        text8 = fp.read()
        # print(text)
    fp.close()

    # tablee8 = make_table(text8, anss8)

    with open('raw_text/Постановление ФАС Уральского округа от 27_07_2012 N Ф09-6215_rzm', "rb") as fp:
        anss9 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 27_07_2012 N Ф09-6215.txt',
             encoding='cp1251', errors='ignore') as fp:
        text9 = fp.read()
        # print(text)
    fp.close()

    # tablee9 = make_table(text9, anss9)

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-4435_rzm', "rb") as fp:
        anss10 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-4435.txt',
             encoding='cp1251', errors='ignore') as fp:
        text10 = fp.read()
        # print(text)
    fp.close()

    # tablee10 = make_table(text10, anss10); плохо сделал именно этот текст

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-4559_rzm', "rb") as fp:
        anss11 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 31_05_2006 N Ф09-4559.txt',
             encoding='cp1251', errors='ignore') as fp:
        text11 = fp.read()
        # print(text)
    fp.close()

    # tablee11

    with open('raw_text/Постановление ФАС Уральского округа от 20_12_2010 N Ф09-9906_rzm', "rb") as fp:
        anss12 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 20_12_2010 N Ф09-9906.txt',
             encoding='cp1251', errors='ignore') as fp:
        text12 = fp.read()
        # print(text)
    fp.close()

    # tablee12

    with open('raw_text/Постановление ФАС Уральского округа от 30_05_2006 N Ф09-4336_rzm', "rb") as fp:
        anss13 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 30_05_2006 N Ф09-4336.txt',
             encoding='cp1251', errors='ignore') as fp:
        text13 = fp.read()
        # print(text)
    fp.close()

    # tablee13

    with open('raw_text/Постановление ФАС Уральского округа от 05_06_2006 N Ф09-205_rzm', "rb") as fp:
        anss14 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 05_06_2006 N Ф09-205.txt',
             encoding='cp1251', errors='ignore') as fp:
        text14 = fp.read()
        # print(text)
    fp.close()

    # tablee14

    with open('raw_text/Постановление ФАС Уральского округа от 30_05_2006 N Ф09-4131_rzm', "rb") as fp:
        anss15 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 30_05_2006 N Ф09-4131.txt',
             encoding='cp1251', errors='ignore') as fp:
        text15 = fp.read()
        # print(text)
    fp.close()

    # tablee15

    with open('raw_text/Постановление ФАС Уральского округа от 06_02_2007 N Ф09-4881_rzm', "rb") as fp:
        anss16 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 06_02_2007 N Ф09-4881.txt',
             encoding='cp1251', errors='ignore') as fp:
        text16 = fp.read()
        # print(text)
    fp.close()

    # tablee16

    with open('raw_text/Постановление ФАС Уральского округа от 30_07_2012 N Ф09-1027_rzm', "rb") as fp:
        anss17 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 30_07_2012 N Ф09-1027.txt',
             encoding='cp1251', errors='ignore') as fp:
        text17 = fp.read()
        # print(text)
    fp.close()

    # tablee17

    with open('raw_text/Постановление ФАС Уральского округа от 01_06_2006 N Ф09-4469_rzm', "rb") as fp:
        anss18 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 01_06_2006 N Ф09-4469.txt',
             encoding='cp1251', errors='ignore') as fp:
        text18 = fp.read()
        # print(text)
    fp.close()

    # tablee18

    with open('raw_text/Постановление ФАС Уральского округа от 20_12_2010 N Ф09-1060_rzm', "rb") as fp:
        anss19 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 20_12_2010 N Ф09-1060.txt',
             encoding='cp1251', errors='ignore') as fp:
        text19 = fp.read()
        # print(text)
    fp.close()

    # tablee19

    with open('raw_text/Постановление ФАС Уральского округа от 01_06_2006 N Ф09-4398_rzm', "rb") as fp:
        anss20 = pickle.load(fp)
        # print(anss)
    fp.close()

    with open('raw_text/Постановление ФАС Уральского округа от 01_06_2006 N Ф09-4398.txt',
             encoding='cp1251', errors='ignore') as fp:
        text20 = fp.read()
        # print(text)
    fp.close()

    # tablee20

    def ohe_tr(onehotencoder, data, categorical_cols, not_fit):
        if not_fit:
            transformed_data = onehotencoder.fit_transform(data[categorical_cols])
        else:
            transformed_data = onehotencoder.transform(data[categorical_cols])
        encoded_data = pd.DataFrame(transformed_data, index=data.index)
        other_data = data.drop(columns=categorical_cols)
        data_out = pd.concat([encoded_data, other_data], axis=1)
        return data_out

    def train_test(n, clf):
        texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10,
                text11, text12, text13, text14, text15, text16, text17, text18, text19,
                text20]
        anss = [anss1, anss2, anss3, anss4, anss5, anss6, anss7, anss8, anss9, anss10,
               anss11, anss12, anss13, anss14, anss15, anss16, anss17, anss18, anss19,
               anss20]

        bigtext = ''
        bigans = []

        for text in texts[:n] + texts[n + 1:]:
            bigtext += text

        for ans in anss[:n] + anss[n + 1:]:
            bigans += ans

        table_train = make_table(bigtext, bigans)
        table_test = make_table(texts[n], anss[n])

        X_train = table_train.drop(['sign index', 'end?'], axis=1)
        y_train = table_train['end?']

        categorical = ['sign', 'wright', 'right', 'left', 'abbleft', 'abbright', 'cright', 'cleft', 'cwright', 'left_caps', 'right_caps']

        X_train_trans = ohe_tr(enc, X_train, categorical, True)

        clf.fit(X_train_trans, y_train)

        X_test = table_test.drop(['sign index', 'end?'], axis=1)
        y_test = table_test['end?']

        X_test_trans = ohe_tr(enc, X_test, categorical, False)
        y_pred = clf.predict(X_test_trans)
        
        return f1_score(y_test, y_pred)

    cnt = 20
    clf1 = DecisionTreeClassifier()
    clf2 = LogisticRegression()
    clf3 = RandomForestClassifier()
    clf4 = GradientBoostingClassifier()
    clf5 = SVC()
    for alg in [clf1, clf2, clf3, clf4, clf4]:
        if alg == clf1:
            print('DecisionTreeClassifier')
        elif alg == clf2:
            print('LogisticRegression')
        elif alg == clf3:
            print('RandomForestClassifier')
        elif alg == clf4:
            print('GradientBoostingClassifier')
        else:
            print('SVC')
        for k in range(0, cnt):
            print(k + 1, '/', cnt, train_test(k, clf))